In [ ]:
import pytesseract 
import requests
import base64
import json
import cv2


In [ ]:

def get_captcha_data():
    captcha_data = None

    url = 'https://gateway-voters.eci.gov.in/api/v1/captcha-service/generateCaptcha' 

    headers = {
    'Accept': 'application/json, text/plain, */*',
    'Origin': 'https://electoralsearch.eci.gov.in',
    'appName': 'ELECTORAL_SEARCH'
    }
    
    try:
        response = requests.get(url, headers=headers)
    

        data = json.loads(response.text)

        if data['status'] == 'Success' and data['statusCode'] == 200:
            captcha_data = {
                'captcha': data['captcha'],
                'id': data['id']
            }
            # print(captcha_data)
        else:
            print("Invalid response")
    except Exception as e:
        print(str(e))
        return None    
    return captcha_data    

In [ ]:
def get_code(base64_image):

    # Decode the base64 string
    image_data = base64.b64decode(base64_image)

    # Write the binary data to a file
    with open('captcha_image.jpg', 'wb') as file:
        file.write(image_data)

    print("Image saved as captcha_image.jpg")

    img = cv2.imread('captcha_image.jpg') 
    text = pytesseract.image_to_string(img)

    return text.lower()


In [ ]:

def get_voter_info(epicno, captcha_id, captcha_data):
    url = 'https://gateway.eci.gov.in/api/v1/elastic/search-by-epic-from-national-display'

    headers = {
        'Accept': 'application/json, text/plain, */*',
        'Content-Type': 'application/json',
        'Origin': 'https://electoralsearch.eci.gov.in',
        'applicationName': 'ELECTORAL_SEARCH',
    }

    data = {
        "isPortal": True,
        "epicNumber": epicno,
        "captchaId": captcha_id,
        "captchaData": captcha_data,
        "securityKey": "na"
    }

    print(data)

    try:
        response = requests.post(url, headers=headers, json=data)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to get data. Status code: {response.text }")
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None



In [ ]:
def get_user_info(epicno):
    got_data = False
    while got_data == False:
        captcha_data = get_captcha_data()

        captcha = captcha_data['captcha']
        captcha_id = captcha_data['id']

        captcha_data = get_code(captcha).split("\n")[0]

        print(captcha_data)


        print(epicno, captcha_id, captcha_data)

        data = get_voter_info(epicno, captcha_id, captcha_data)
        if data != None:
            got_data = True
            print(data)
        else:
            print("wrong captcha")    

In [ ]:
get_user_info("epicnumber")